In [3]:
import pandas as pd
import numpy as np
import os
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from torch.utils.data.dataset import random_split


In [11]:
train_csv_path = "../ondemand/train.csv"

In [12]:
train_imgs_dir = "../ondemand/train_feature_maps_aug/"

In [13]:
label_dict = {'HGSC':0, 'EC':1, 'CC':2, 'LGSC':3, 'MC':4}

In [14]:
revlabel_dict = {v:k for k,v in label_dict.items() }

In [16]:
class OvarianCancerFtrDataset(Dataset):

    def __init__(self, csv_file, root_dir, label_dict, transform=None):

        self.df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.label_dict = label_dict
        self.image_fnames = os.listdir(root_dir)

    def __len__(self):
        return len(self.image_fnames)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = os.path.join(self.root_dir, str(self.image_fnames[idx]))
        image = torch.load(img_path)
        label = self.df.loc[self.df['image_id'] == int(self.image_fnames[idx].split('.')[0].split('_')[0])]['label'].iloc[0]
        label = self.label_dict[label]

        if self.transform:
            image = self.transform(image)

        return image, label

In [17]:
dataset = OvarianCancerFtrDataset(train_csv_path, train_imgs_dir, label_dict)

In [20]:
train_size = int(0.8 * len(dataset))  
val_size = len(dataset) - train_size  
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)


In [63]:
class ClassificationModel(nn.Module):
    def __init__(self, num_classes):
        super(ClassificationModel, self).__init__()
        self.fc1 = nn.Linear(2048, 64) 
        self.fc2 = nn.Linear(64, 32) 
        self.fc3 = nn.Linear(32, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.softmax(self.fc3(x), dim = 1)
        return x


In [93]:
model = ClassificationModel(num_classes = 5)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)  


In [94]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device = torch.device("cpu")

model = model.to(device)


In [95]:
num_epochs = 50

for epoch in range(num_epochs):
    
    model.train()
    total_train_loss = 0

    for batch in tqdm(train_loader):
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device) 

        outputs = model(inputs.float())
        loss = criterion(outputs, labels.long())  

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)

    model.eval() 
    total_val_loss = 0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad(): 
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device) 

            outputs = model(inputs.float())
            loss = criterion(outputs, labels.long())

            total_val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    avg_val_loss = total_val_loss / len(val_loader)
    val_accuracy = correct_predictions / total_predictions

    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')


100%|███████████████████████████████████████████████████████████████| 860/860 [00:02<00:00, 424.94it/s]


Epoch [1/50], Training Loss: 1.4995, Validation Loss: 1.4770, Validation Accuracy: 0.4163


100%|███████████████████████████████████████████████████████████████| 860/860 [00:00<00:00, 927.84it/s]


Epoch [2/50], Training Loss: 1.4719, Validation Loss: 1.4484, Validation Accuracy: 0.4500


100%|███████████████████████████████████████████████████████████████| 860/860 [00:00<00:00, 944.67it/s]


...

Epoch [48/50], Training Loss: 1.2386, Validation Loss: 1.2452, Validation Accuracy: 0.6663


100%|███████████████████████████████████████████████████████████████| 860/860 [00:00<00:00, 911.78it/s]


Epoch [49/50], Training Loss: 1.2369, Validation Loss: 1.2557, Validation Accuracy: 0.6500


100%|███████████████████████████████████████████████████████████████| 860/860 [00:00<00:00, 927.06it/s]


Epoch [50/50], Training Loss: 1.2330, Validation Loss: 1.2396, Validation Accuracy: 0.6733


In [28]:
torch.save(model.state_dict(), 'model_acc_67aug.pt')
